In [ ]:
## If using Google colab
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)


In [1]:
import pandas as pd
import io
import numpy as np
import functools
import operator
import regex as re

# Set up read path, read file, write path, write file

In [2]:
curr_project_name = "project_erudition_emails_2"
notebook_path = "/mnt/c/Users/isaac/Desktop/github_mturks_assistant/mturks_assistant/notebooks"

input_filename = 'Batch_4551511_batch_results.xlsx'
read_path = notebook_path+"/../projects/"+curr_project_name+"/8_mOut_leads/"+input_filename

write_path_generic = notebook_path+"/../projects/"+curr_project_name+"/9_notebookProcessed_leads/"
output_filename_leadtype = 'cleaned_mturk_emails.xlsx'
write_path_leadtype = notebook_path+"/../projects/"+curr_project_name+"/9_notebookProcessed_leads/"+output_filename_leadtype

output_filename_leadtype_turks = 'mturk_checker_emails.xlsx'
write_path_turks = notebook_path+"/../projects/"+curr_project_name+"/9_notebookProcessed_leads/"+output_filename_leadtype_turks


# Read in MTurks dataframe and do preliminary reformatting and unstacking of data

In [3]:
df0 = pd.read_excel(read_path)
pd.set_option('display.max_columns',1000)

# # add the anchor presence to the  dataframe - scratch work
# prefix = "Answer."
# item_column = "email_1"
# df0['HIT_anchor'] = -1
# i=1
# for index, row in df0.iterrows():
#   anchor_presence = int(bool(re.search('[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}', str(df0.loc[index,prefix+item_column]), re.IGNORECASE)))
#   df0.loc[index, 'HIT_anchor'] = anchor_presence

# df0.loc[df0['HIT_anchor'] == 0]['Answer.email_1']="N/A_0"

In [4]:
# Check and clean nan values
print('df0 nan values: ',df0.isnull().sum().sum())
df1 = df0.fillna('N/A_1')
print('df1 nan values: ',df1.isnull().sum().sum())

# Unstacking of the data side by side 
df1['copy_id'] = -1
df1=df1.reset_index()

# # Group by input columns
# input_cols = [col for col in df1.columns if 'Input' in col]
# dfgroups = df1.groupby(input_cols).size().reset_index().rename(columns={0:'count'})
# Group by HITId
dfgroups = df1.groupby(['HITId']).size().reset_index().rename(columns={0:'count'})

for row_num, row in dfgroups.iterrows():
  #targets = [row[col] for col in input_cols]
  #filtered_dfs = (df1[input_cols[index]]== target for index, target in enumerate(targets))
  #df_curr_group = df1.loc[functools.reduce(operator.and_, filtered_dfs)]

  df_curr_group = df1.loc[ df1['HITId'] == row['HITId']]

  ctr = 1
  for row_ix, group in df_curr_group.iterrows():
    df1.at[row_ix,'copy_id'] = ctr
    ctr += 1

# Check and clean nan values
print('df1 nan values: ',df1.isnull().sum().sum())
df2 = df1.set_index(['HITId','copy_id'])
print('df2 nan values: ',df2.isnull().sum().sum())
#df2.head()

df3 = df2.unstack('copy_id')    # you won't always have perfect redundancy so, that will introduce NAN values
# df3.head()

# Check and clean nan values
print('df3 nan values: ',df3.isnull().sum().sum())
df4 = df3.fillna('N/A_2')
print('df4 nan values: ',df4.isnull().sum().sum())

#adjust multiindex level into one level
df4.columns = ['_'.join(map(str,col)) for col in df4.columns.values]

# Check and clean nan values
print('df4 nan values: ',df4.isnull().sum().sum())
df5 = df4.reset_index()
print('df5 nan values: ',df5.isnull().sum().sum())

df5.head()

# Removing all unnecessary columns for coming analysis
# KEEP ALL INPUT DATA FOR LATER CHECKING (all inputs, all answers, hitid, workerid)

relevant_columns = ['HITId'] + [col for col in df5.columns if 'WorkerId' in col] + [col for col in df5.columns if 'ApprovalRate' in col] + [col for col in df5.columns if 'Input' in col] + [col for col in df5.columns if 'Answer' in col]
df6 = df5[relevant_columns]

# Check and clean nan values
print('df6 nan values: ',df6.isnull().sum().sum())
df7 = df6.fillna('N/A_3')
print('df7 nan values: ',df7.isnull().sum().sum())

# ["-","","n/a","na",'N/A_1', 'no'] --> you need to convert everything like this to one thing

df0 nan values:  28918
df1 nan values:  0
df1 nan values:  0
df2 nan values:  0
df3 nan values:  3485
df4 nan values:  425
df4 nan values:  425
df5 nan values:  425
df6 nan values:  0
df7 nan values:  0


# Evaluate worker quality

In [46]:
df_workers = pd.DataFrame(columns=['WorkerId', 'total_contribution', 'answers','answer_counts'])

workerGroups = df0.groupby(['WorkerId']).size().reset_index().rename(columns={0:'count'})
for row_num, row in workerGroups.iterrows():
  curr = df0.loc[df0['WorkerId']==row.WorkerId]
  curr_new = curr.shape[0]

  # count number of non "N/A" repeats
  duplicates = curr[curr.duplicated(subset='Answer.email_1',keep=False)]
  uniques = curr['Answer.email_1'].unique()

  if duplicates.shape[0] >= 1:
    print('issue')
    print(duplicates)

    workerId = row['WorkerId']
    print(workerId)

    print(duplicates['Answer.email_1'].value_counts())
    answers = list(duplicates['Answer.email_1'].value_counts().index)
    answers_string = "[" + ", ".join(answers) +"]"
    answer_counts = list(duplicates['Answer.email_1'].value_counts())
    answer_counts_string = "[" + ", ".join([str(x) for x in answer_counts]) +"]"

    print(answers_string)
    print(answer_counts_string)


    print("duplicates shape: ",str(duplicates.shape[0]))
    print("uniques shape: ", str(uniques.shape[0]))
    print("total contribution: ",str(curr.shape[0]))

    total_contribution = curr.shape[0]
    unique_contributions = uniques.shape[0]
    duplicate_contributions = duplicates.shape[0]

    df_workers.loc[len(df_workers.index)] = [workerId, total_contribution, answers_string, answer_counts_string] 


issue
                               HITId                       HITTypeId  \
1127  3LN3BXKGD7HW5IWRY7T9O5M68YMGW4  3VILS635XN3T15HZBEFB54Q50VJPSD   
1139  307FVKVSZY113N26VYJWN0QMP24473  3VILS635XN3T15HZBEFB54Q50VJPSD   

                                                  Title  \
1127  Gather contact information for the student clu...   
1139  Gather contact information for the student clu...   

                                            Description  \
1127  Gather all available contact information for a...   
1139  Gather all available contact information for a...   

                                               Keywords  Reward  \
1127  data collection, data extraction, student club...    0.03   
1139  data collection, data extraction, student club...    0.03   

                      CreationTime  MaxAssignments  \
1127  Thu Sep 09 11:42:04 PDT 2021               3   
1139  Thu Sep 09 11:42:05 PDT 2021               3   

                                   RequesterAnnotation  

In [ ]:
# Create list of workers to ignore/block or not to trust - from manual inspection
workers_to_flag = []

# Create labelling for the leads dataframe

In [7]:
df_leads_processing = df7.copy()

lead_groups = [['email_1','email_2'],['facebook'],['instagram'],['phone_number'],['website']]
lead_groups_anchors = [['^[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}$','^[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}$'],['facebook\.com'],['^(@?)[a-zA-Z0-9_][a-zA-Z0-9_.]{2,29}'],['[0-9]{3,11}'],['\.(com|org|net|edu)']]
redundancy = 3 # the redundancy is built into this it will take alot of work to automate this for more or less redundancy
prefix = 'Answer.'

# sameness_list = ['answer_sameness_'+str(x) for x in [1,2,3]]
# anchor_list = ['answer_anchor_'+str(x) for x in [1,2,3]]
# decision_list = ['answer_decision_'+str(x) for x in [1,2,3]]
# others_list = ['cumulative_sameness','cumulative_anchor','best_guess','runners_up'] # one day you may do a priority ranker
# df_leads_processing[sameness_list] = -1 # will be 1/redundancy up to redundancy
# df_leads_processing[anchor_list] = -1 # will be 1 or 0
# df_leads_processing[decision_list] = -1 # will be 1 or 0 if/when visited
# df_leads_processing[others_list] = [-1,-1,'_','_'] 

# Create labelling for the HITs based on the presence of a regex anchor, and their "sameness" with respect to the redundant HITs

for lgi in range(len(lead_groups)):
  lead_group = lead_groups[lgi]
  lead_group_anchors = lead_groups_anchors[lgi]

  for i in range(len(lead_group)):
    item_column = lead_group[i]
    anchor = lead_group_anchors[i]
    
    df_leads_processing[item_column+"_best_guess"] = "N/A_4"
    df_leads_processing[item_column+"_runners_up"] = "N/A_4"

    for index, row in df_leads_processing.iterrows():

      # Make the anchor presence labelling for current item_column
      anchor_sum = 0
      for i in [1,2,3]: # from redundancy
        print(prefix+item_column+str(i))
        print(df_leads_processing.loc[index,prefix+item_column+"_"+str(i)])
        anchor_presence = int(bool(re.fullmatch(anchor, \
            str(df_leads_processing.loc[index,prefix+item_column+"_"+str(i)]), re.IGNORECASE))) 
        df_leads_processing.loc[index, item_column+'_answer_anchor_'+str(i)] = anchor_presence
        anchor_sum+=anchor_presence
      df_leads_processing.loc[index, item_column+'_cumulative_anchor'] = anchor_sum

      # Make the sameness labelling 
      # This currently only works for redundancy = 3, you have to add sameness group labels for groups larger than 3
      temp_dict = {} # (answer_string:sameness)
      answer_col_list = [prefix+item_column+"_"+str(i) for i in [1,2,3]]
      str_list = row[answer_col_list]
      for item in str_list:
        temp_dict[item]=0
      for item in str_list:
        temp_dict[item]+=1
      sameness_sum = 0
      for i in [1,2,3]:
        col = answer_col_list[i-1]
        val = temp_dict[row[col]]
        df_leads_processing.loc[index, item_column+'_answer_sameness_'+str(i)] = val
        sameness_sum += val
      df_leads_processing.loc[index, item_column+'_cumulative_sameness'] = sameness_sum
      

      # Based on anchor presence and "sameness" values, autoassign the "best_guess" value and "runners_up" values
        # Any viable HIT answers that don't have the maximum sameness value are added to the runners up list

        # Need the list of answer_column names
        # Need the list of answer_column string values
        # Need the list of answer_column sameness values
        # Need the list of answer_column anchor values
      answer_col_list = [prefix+item_column+"_"+str(i) for i in [1,2,3]]
      answer_list = [df_leads_processing.loc[index,col] for col in answer_col_list]
      sameness_list = [df_leads_processing.loc[index,item_column+"_answer_sameness_"+str(i)] for i in [1,2,3]]
      anchor_list = [df_leads_processing.loc[index,item_column+"_answer_anchor_"+str(i)] for i in [1,2,3]]
      sameness_list, answer_col_list, answer_list, anchor_list = zip(*sorted(zip(sameness_list, answer_col_list, answer_list,anchor_list),reverse=True))


      flag = False
      runners_up = []
      visited_answers = []
      for i in range(len(sameness_list)):
        curr_answer_toCheck = answer_list[i]
        curr_answer_anchor = anchor_list[i]

        if not flag and curr_answer_anchor == 1:
          df_leads_processing.loc[index,item_column+"_best_guess"] = curr_answer_toCheck
          visited_answers.append(curr_answer_toCheck)
          flag = True
        elif flag and (curr_answer_anchor == 1) and (curr_answer_toCheck not in visited_answers):         
            runners_up.append(curr_answer_toCheck)
            visited_answers.append(curr_answer_toCheck)
      if len(runners_up)>0:
        df_leads_processing.loc[index,item_column+"_runners_up"] = ",".join([str(x) for x in runners_up])



Answer.email_11
SSCHLESMAN@rollins.edu
Answer.email_12
N/A_1
Answer.email_13
SSCHLESMAN@rollins.edu
Answer.email_11
womeninmedicine1989@gmail.com
Answer.email_12
womeninmedicine1989@gmail.com
Answer.email_13
womeninmedicine1989@gmail.com
Answer.email_11
Page error
Answer.email_12
-
Answer.email_13
N/A_1
Answer.email_11
dentalclub@pfw.edu
Answer.email_12
dentalclub@pfw.edu
Answer.email_13
N/A_1
Answer.email_11
N/A_1
Answer.email_12
adrollet@uvm.edu
Answer.email_13
adrollet@uvm.edu
Answer.email_11
studentlife@rmu.edu
Answer.email_12
N/A_1
Answer.email_13
studentlife@rmu.edu
Answer.email_11
tmm004@aquinas.edu
Answer.email_12
None - asks for submitter's information
Answer.email_13
N/A_1
Answer.email_11
iusdasda@iupui.edu
Answer.email_12
iusdasda@iupui.edu
Answer.email_13
emwelsh@wisc.edu
Answer.email_11
-
Answer.email_12
N/A_1
Answer.email_13
Nonr
Answer.email_11
jkowolik@iu.edu
Answer.email_12
jkowolik@iu.edu
Answer.email_13
jkowolik@iu.edu
Answer.email_11
ucsd.aed@gmail.com
Answer.email_

In [8]:
df_leads_processed = df_leads_processing[['HITId','WorkerId_1'] + [col for col in df_leads_processing.columns if 'Input' in col and '_1' in col] + [col for col in df_leads_processing.columns if "best_guess" in col] + [col for col in df_leads_processing.columns if "runners_up" in col]  + [col for col in df_leads_processing.columns if "cumulative" in col]]

# Create the final, leads list for each of the lead types (for email will need to connect the two emails columns together), making the database you need (and keeping HITIds, AssignmentIds, and WorkerIds to link the data with other data later)

In [9]:
df_leads_processing.head()

,HITId,WorkerId_1,WorkerId_2,WorkerId_3,LifetimeApprovalRate_1,LifetimeApprovalRate_2,LifetimeApprovalRate_3,Last30DaysApprovalRate_1,Last30DaysApprovalRate_2,Last30DaysApprovalRate_3,Last7DaysApprovalRate_1,Last7DaysApprovalRate_2,Last7DaysApprovalRate_3,Input.university_1,Input.university_2,Input.university_3,Input.city_1,Input.city_2,Input.city_3,Input.country_1,Input.country_2,Input.country_3,Input.directory_link_1,Input.directory_link_2,Input.directory_link_3,Input.club_name_1,Input.club_name_2,Input.club_name_3,Input.club_link_1,Input.club_link_2,Input.club_link_3,Answer.email_1_1,Answer.email_1_2,Answer.email_1_3,Answer.email_2_1,Answer.email_2_2,Answer.email_2_3,Answer.facebook_1,Answer.facebook_2,Answer.facebook_3,Answer.instagram_1,Answer.instagram_2,Answer.instagram_3,Answer.phone_number_1,Answer.phone_number_2,Answer.phone_number_3,Answer.website_1,Answer.website_2,Answer.website_3,email_1_best_guess,email_1_runners_up,email_1_answer_anchor_1,email_1_answer_anchor_2,email_1_answer_anchor_3,email_1_cumulative_anchor,email_1_answer_sameness_1,email_1_answer_sameness_2,email_1_answer_sameness_3,email_1_cumulative_sameness,email_2_best_guess,email_2_runners_up,email_2_answer_anchor_1,email_2_answer_anchor_2,email_2_answer_anchor_3,email_2_cumulative_anchor,email_2_answer_sameness_1,email_2_answer_sameness_2,email_2_answer_sameness_3,email_2_cumulative_sameness,facebook_best_guess,facebook_runners_up,facebook_answer_anchor_1,facebook_answer_anchor_2,facebook_answer_anchor_3,facebook_cumulative_anchor,facebook_answer_sameness_1,facebook_answer_sameness_2,facebook_answer_sameness_3,facebook_cumulative_sameness,instagram_best_guess,instagram_runners_up,instagram_answer_anchor_1,instagram_answer_anchor_2,instagram_answer_anchor_3,instagram_cumulative_anchor,instagram_answer_sameness_1,instagram_answer_sameness_2,instagram_answer_sameness_3,instagram_cumulative_sameness,phone_number_best_guess,phone_number_runners_up,phone_number_answer_anchor_1,phone_number_answer_anchor_2,phone_number_answer_anchor_3,phone_number_cumulative_anchor,phone_number_answer_sameness_1,phone_number_answer_sameness_2,phone_number_answer_sameness_3,phone_number_cumulative_sameness,website_best_guess,website_runners_up,website_answer_anchor_1,website_answer_anchor_2,website_answer_anchor_3,website_cumulative_anchor,website_answer_sameness_1,website_answer_sameness_2,website_answer_sameness_3,website_cumulative_sameness
0,301KG0KXAJ7OTYTV4TMKLYIIJR4H2M,A1WOSUFQG768EF,A3H70RJNKHWKBD,A3U5D83UGFY23Y,100% (660/660),98% (3984/4075),100% (394/394),0% (0/0),0% (0/0),0% (0/0),0% (0/0),0% (0/0),0% (0/0),Rollins College,Rollins College,Rollins College,Winter Park,Winter Park,Winter Park,United States,United States,United States,https://rollins.campuslabs.com/engage/organiza...,https://rollins.campuslabs.com/engage/organiza...,https://rollins.campuslabs.com/engage/organiza...,Rollins Pre-Dental Program,Rollins Pre-Dental Program,Rollins Pre-Dental Program,https://rollins.campuslabs.com/engage/organiza...,https://rollins.campuslabs.com/engage/organiza...,https://rollins.campuslabs.com/engage/organiza...,SSCHLESMAN@rollins.edu,N/A_1,SSCHLESMAN@rollins.edu,N/A_1,N/A_1,N/A_1,N/A_1,N/A_1,N/A_1,N/A_1,N/A_1,N/A_1,727-510-6912,N/A_1,727-510-6912,N/A_1,N/A_1,N/A_1,SSCHLESMAN@rollins.edu,N/A_4,1.0,0.0,1.0,2.0,2.0,1.0,2.0,5.0,N/A_4,N/A_4,0.0,0.0,0.0,0.0,3.0,3.0,3.0,9.0,N/A_4,N/A_4,0.0,0.0,0.0,0.0,3.0,3.0,3.0,9.0,N/A_4,N/A_4,0.0,0.0,0.0,0.0,3.0,3.0,3.0,9.0,N/A_4,N/A_4,0.0,0.0,0.0,0.0,2.0,1.0,2.0,5.0,N/A_4,N/A_4,0.0,0.0,0.0,0.0,3.0,3.0,3.0,9.0
1,302OLP89E6TMHTDJVG1YGP40INRACD,A1WI8RUR1D2MQI,A23UEV7V1QKJWW,AS90M5SRFWQ82,0% (0/0),0% (0/0),100% (82/82),0% (0/0),0% (0/0),0% (0/0),0% (0/0),0% (0/0),0% (0/0),The University of Tampa,The University of Tampa,The University of Tampa,Tampa,Tampa,Tampa,United States,United States,United States,https://utexas.campuslabs.com/engage/organizat...,https://utexas.campuslabs.com/engage/organizat...,https://utexas.campuslabs.com/engage/organiza

In [10]:
# df_emails_large = pd.DataFrame(columns=['university','club','winner_email','flag','list_id'])
lead_group_names = ['email','facebook','instagram','phone number','website']
lead_groups = [['email_1','email_2'],['facebook'],['instagram'],['phone_number'],['website']]
df_leads_list = []
df_fileName_list = []
blatest = {}

for lgi in range(len(lead_groups)):
  input_cols = [col for col in df_leads_processed.columns if "Input" in col]

  lead_group = lead_groups[lgi]
  lead_group_name = lead_group_names[lgi]
  lead_group_cols = [lead_group_name+"_best_guess",lead_group_name+"_runners_up", lead_group_name+"_cumulative_sameness", lead_group_name+"_cumulative_anchor"]
  lead_group_df_cols = ['HITId','WorkerId_1'] + input_cols + lead_group_cols + ["lead_subgroup"]


  # print(input_cols)
  # print(lead_group)
  # print(lead_group_name)
  print(lead_group_df_cols)
  print("\n\n")

  lead_group_df = pd.DataFrame(columns=lead_group_df_cols)
  print("leadgroupdfshape1")
  print(lead_group_df.shape)

  # you're gonna loop through all email lead_subgroups
  for lead_subgroup in lead_group:
    # df_leads_processed_modded = df_leads_processed.rename(columns = lead_group_df_cols)

    # print('lead_subgroup: ',lead_subgroup)
    lead_subgroup_name = lead_subgroup
    # lead_subgroup_name = 
    lead_subgroup_cols = [lead_subgroup_name+"_best_guess",lead_subgroup_name+"_runners_up", lead_subgroup_name+"_cumulative_sameness", lead_subgroup_name+"_cumulative_anchor"]
    lead_subgroup_df_cols = ['HITId','WorkerId_1'] + input_cols + lead_subgroup_cols
    lead_subgroup_df = df_leads_processed[lead_subgroup_df_cols]
    lead_subgroup_df['lead_subgroup'] = lead_subgroup_name
    lead_subgroup_df.columns = lead_group_df_cols

    lead_group_df = pd.concat([lead_group_df, lead_subgroup_df],  axis=0)
    print("leadgroupdfshapeupdate")
    print(lead_group_df.shape)

  df_fileName_list.append(write_path_generic+"/"+"leads_"+lead_group_name+".xlsx")
  df_leads_list.append(lead_group_df)
#  print(lead_group_df)

'''
    print('issue')
    print(row[('results','winner_email_1')])
    print('index: ',index)
    print('\n')
'''
    # print(row[('results','winner_email_1')])

  # df_emails_large = df_emails_large.fillna('N/A')
  # df_emails_large = df_emails_large.loc[df_emails_large['winner_email']!='N/A']
  # leads_df_dicts[lead_group_name] = curr_df

['HITId', 'WorkerId_1', 'Input.university_1', 'Input.city_1', 'Input.country_1', 'Input.directory_link_1', 'Input.club_name_1', 'Input.club_link_1', 'email_best_guess', 'email_runners_up', 'email_cumulative_sameness', 'email_cumulative_anchor', 'lead_subgroup']



leadgroupdfshape1
(0, 13)
leadgroupdfshapeupdate
(909, 13)
leadgroupdfshapeupdate
(1818, 13)
['HITId', 'WorkerId_1', 'Input.university_1', 'Input.city_1', 'Input.country_1', 'Input.directory_link_1', 'Input.club_name_1', 'Input.club_link_1', 'facebook_best_guess', 'facebook_runners_up', 'facebook_cumulative_sameness', 'facebook_cumulative_anchor', 'lead_subgroup']



leadgroupdfshape1
(0, 13)
leadgroupdfshapeupdate
(909, 13)
['HITId', 'WorkerId_1', 'Input.university_1', 'Input.city_1', 'Input.country_1', 'Input.directory_link_1', 'Input.club_name_1', 'Input.club_link_1', 'instagram_best_guess', 'instagram_runners_up', 'instagram_cumulative_sameness', 'instagram_cumulative_anchor', 'lead_subgroup']



leadgroupdfshape1
(0, 13)

<ipython-input-10-de9c8d8f234d>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lead_subgroup_df['lead_subgroup'] = lead_subgroup_name


"\n    print('issue')\n    print(row[('results','winner_email_1')])\n    print('index: ',index)\n    print('\n')\n"

## Push the runners up data into the leads directories for each dataframe and also identify redundancies to look at

In [54]:
df_leads_with_runnersUp_list = []
for i in range(len(df_leads_list)):
  lead_group_name = lead_group_names[i]
  df_curr = df_leads_list[i].copy()
  print(df_curr.columns)
  for index, row in df_curr.iterrows():
    rustring = row[lead_group_name+"_runners_up"]
    runners_up_list = rustring.split(',')

    # if len(runners_up_list) > 0:
    if "N/A_4" not in runners_up_list: 
      for ru in runners_up_list:
        print(ru)
        # make a copy of the current row
        curr_row_copy = row.copy()
        curr_row_copy[lead_group_name+"_best_guess"] = ru
        df_curr.loc[len(df_curr.index)] = curr_row_copy
        print("curr row shape: ",str(curr_row_copy.shape))
        print("df shape: ", str(df_curr.shape))

  df_curr[lead_group_name+'_runners_up'] = ''
  df_curr['duplicate_lead'] = 0
  df_curr.loc[df_curr.duplicated(subset=lead_group_name+'_best_guess',keep=False),'duplicate_lead'] = 1
  
  df_leads_with_runnersUp_list.append(df_curr)
    # for index, row in df_curr.iterrows():
    #   row
df_leads_with_runnersUp_list[0].head()

Index(['HITId', 'WorkerId_1', 'Input.university_1', 'Input.city_1',
       'Input.country_1', 'Input.directory_link_1', 'Input.club_name_1',
       'Input.club_link_1', 'email_best_guess', 'email_runners_up',
       'email_cumulative_sameness', 'email_cumulative_anchor',
       'lead_subgroup'],
      dtype='object')
emwelsh@wisc.edu
curr row shape:  (13,)
df shape:  (1819, 13)
stephanie.authement@nicholls.edu
curr row shape:  (13,)
df shape:  (1820, 13)
sjcollae@mtu.edu
curr row shape:  (13,)
df shape:  (1821, 13)
cassidy.hyde@ypal.org
curr row shape:  (13,)
df shape:  (1822, 13)
Stephanie.purcell@unmc.edu
curr row shape:  (13,)
df shape:  (1823, 13)
dentalawarenessusf@gmail.com
curr row shape:  (13,)
df shape:  (1824, 13)
BYUHpredentalclub@gmail.com
curr row shape:  (13,)
df shape:  (1825, 13)
emwelsh@wisc.edu
curr row shape:  (13,)
df shape:  (1826, 13)
csthomas@alasu.edu
curr row shape:  (13,)
df shape:  (1827, 13)
admission@cuaa.edu
curr row shape:  (13,)
df shape:  (1828, 13)
adm

,HITId,WorkerId_1,Input.university_1,Input.city_1,Input.country_1,Input.directory_link_1,Input.club_name_1,Input.club_link_1,email_best_guess,email_runners_up,email_cumulative_sameness,email_cumulative_anchor,lead_subgroup,duplicate_lead
0,301KG0KXAJ7OTYTV4TMKLYIIJR4H2M,A1WOSUFQG768EF,Rollins College,Winter Park,United States,https://rollins.campuslabs.com/engage/organiza...,Rollins Pre-Dental Program,https://rollins.campuslabs.com/engage/organiza...,SSCHLESMAN@rollins.edu,,5.0,2.0,email_1,0
1,302OLP89E6TMHTDJVG1YGP40INRACD,A1WI8RUR1D2MQI,The University of Tampa,Tampa,United States,https://utexas.campuslabs.com/engage/organizat...,Women in Medicine,https://utexas.campuslabs.com/engage/organizat...,womeninmedicine1989@gmail.com,,9.0,3.0,email_1,0
2,302OLP89E6TMHTDJVG1YGP40INRCAF,A1A9GCQVBEMT58,Daytona State College,Daytona Beach,United States,https://www.daytonastate.edu/student_act/clubs...,Student American Dental Hygienists Association...,daytonastate.edu/student_act/clubsandorganizat...,N/A_4,,3.0,0.0,email_1,1
3,302U8RURK6NP0LD8V118SNCP216NV9,A1K0S6EMDIWGWI,Purdue University Fort Wayne,Fort Wayne,United States,https://www.pfw.edu/student-life/student-organ...,Dental Club,https://www.pfw.edu/student-life/student-organ...,dentalclub@pfw.edu,,5.0,2.0,email_1,1
4,304QEQWK0W6RP5C4TJDWMJM4YFY0O9,A3H70RJNKHWKBD,University of Vermont,Burlington,United States,https://clubs.uvm.edu/organizations,Pre-Dental Society,https://clubs.uvm.edu/organization/predental-s...,adrollet@uvm.edu,,5.0,2.0,email_1,1


## Export the dataframes

In [55]:
for i in range(len(df_leads_list)):
  # df_leads_list[i].to_excel(df_fileName_list[i])
  df_leads_with_runnersUp_list[i].to_excel(df_fileName_list[i])

df_workers.to_excel(write_path_turks)
df_leads_processed.to_excel(write_path_leadtype)